In [ ]:
%pip install semantic-link-labs

In [ ]:
import sempy_labs as labs
from sempy_labs.directlake._get_shared_expression import get_shared_expression
from sempy_labs.tom import connect_semantic_model
from sempy_labs import report
import json

In [ ]:
lakehouse_name  =  "FabricDocs"
lakehouse  =  mssparkutils.lakehouse.get(lakehouse_name)
abfsPath = lakehouse.properties["abfsPath"]
workspace_name  =  notebookutils.runtime.context.get("currentWorkspaceName") # Get current workspace name

In [ ]:
### Create new semantic model
semantic_model_name  =  f"{lakehouse_name}_Model"
labs.create_blank_semantic_model(semantic_model_name)

In [ ]:
### Add tables, columns, hierarchies and measures
lakehouse_workspace = lakehouse.workspaceId
shEx = get_shared_expression(lakehouse_name, lakehouse_workspace)

tables = {
    "FabricRestApiDocs": {
        "display_name": "Fabric REST APIs",
        "columns": {
            "Title": {"source_column": "Title", "data_type": "string" },
            "Service": {"source_column": "Service", "data_type": "string" },
            "API": {"source_column": "API", "data_type": "string" },
            "Article Url": {"source_column": "ArticleUrl", "data_type": "string", "data_category": "WebURL"},
            "Supports User": {"source_column": "SupportUserIdentity", "data_type": "string" },
            "Supports Service principal & Managed Identities": {"source_column": "SupportSpnAndMi", "data_type": "string" }
        },
        "hierarchies" : {
            "Service hierarchy": {"columns": ["Service", "API", "Title"] }
        }

    },
    "SupportedIdentities": {
        "display_name": "Supported Identities",
        "columns": {
            "IdentityNo": {"source_column": "IdentityNo", "data_type": "integer", "hidden": True },
            "Identity": {"source_column": "Identity", "data_type": "string", "sort_by" : "IdentityNo" },
            "Identity Description": {"source_column": "Identity_desc", "data_type": "string" },
            "Identity Support Article Url": {"source_column": "IdentitySupportArticle", "data_type": "string", "data_category": "WebURL" }
        }
    }
}

with connect_semantic_model(dataset=semantic_model_name, readonly=False, workspace=workspace_name) as tom:
    if not any(e.Name == "DatabaseQuery" for e in tom.model.Expressions):
        tom.add_expression("DatabaseQuery", expression=shEx)
        print(f"The 'DatabaseQuery' expression has been added.")

    for table_name, table_data in tables.items():
        display_name = table_data["display_name"]
        columns = table_data.get("columns", None)
        hierarchies = table_data.get("hierarchies", None)
        
        print(f"Processing table: {table_name} ({display_name})")
        if not any(t.Name == display_name for t in tom.model.Tables):
            tom.add_table(name = display_name)
            tom.add_entity_partition(table_name = display_name, entity_name = table_name)
            print(f"The '{display_name}' table has been added.")
        
        for column_name, attributes in columns.items():
            source_column = attributes["source_column"]
            data_type = attributes["data_type"]
            is_hidden = attributes.get("hidden", False)
            data_category = attributes.get("data_category", None)
            sort_by = attributes.get("sort_by", None)
            
            if not any(c.Name == column_name and c.Parent.Name == display_name for c in tom.all_columns()):
                tom.add_data_column(
                    table_name = display_name,
                    column_name = column_name,
                    source_column = source_column,
                    hidden = is_hidden,
                    data_type = data_type,
                    data_category = data_category,
                )
                if(sort_by is not None):
                    tom.set_sort_by_column(table_name = display_name, column_name = column_name, sort_by_column = sort_by)
            
                print(f"The '{source_column}' source column has been added as column '{column_name}'")



        if hierarchies is not None:
            for hierarchy_name, hierarchy_columns in hierarchies.items():
                if not any(h.Name == hierarchy_name for h in tom.all_hierarchies()):
                    h_columns = hierarchy_columns["columns"]
                    tom.add_hierarchy(
                        table_name = display_name,
                        hierarchy_name = hierarchy_name,
                        columns = h_columns
    )

    # Adding measures
    m_expression = """SWITCH(
                        SELECTEDVALUE( 'Supported Identities'[IdentityNo] ),
                        1, CALCULATE( COUNTROWS( 'Fabric REST APIs' ), 'Fabric REST APIs'[Supports User] = "Yes" ),
                        2, CALCULATE( COUNTROWS( 'Fabric REST APIs' ), 'Fabric REST APIs'[Supports Service principal & Managed Identities] = "Yes" ),
                        COUNTROWS( 'Fabric REST APIs' )
                )"""

    rest_api_table = tom.model.Tables["Fabric REST APIs"]
    measure_name = "Supported identity count"
    if not any(m.Name == measure_name for m in tom.all_measures()):
        tom.add_measure(
            table_name = rest_api_table.Name, 
            measure_name = measure_name, 
            expression = m_expression
        )
        print(f"The '{measure_name}' measure has been added to table '{rest_api_table.Name}'")
    else:
        tom.update_measure(
            measure_name = measure_name, 
            expression = m_expression
        )
        print(f"The '{measure_name}' measure has been updated in table '{rest_api_table.Name}'")

    m_expression = """VAR api_cnt = COUNTROWS('Fabric REST APIs')
        RETURN
            IF(
                [Supported identity count] - api_cnt = 0, UNICHAR( 9989 ),
                IF( ISBLANK([Supported identity count]), UNICHAR( 10060 ), UNICHAR( 128993 ))
            )"""

    rest_api_table = tom.model.Tables["Supported Identities"]
    measure_name = "Identity is supported"
    if not any(m.Name == measure_name for m in tom.all_measures()):
        tom.add_measure(
            table_name = rest_api_table.Name, 
            measure_name = measure_name, 
            expression = m_expression
        )
        print(f"The '{measure_name}' measure has been added to table '{rest_api_table.Name}'")
    else:
        tom.update_measure(
            measure_name = measure_name, 
            expression = m_expression
        )
        print(f"The '{measure_name}' measure has been updated in table '{rest_api_table.Name}'")

In [ ]:
# Read the file as a DataFrame where each row represents a line in the file. 
# NOTE: Remember to attach the lakehouse containing the report.json file.
df = spark.read.text(f"{abfsPath}/Files/report.json")

# Convert the DataFrame rows (lines) into a single string
json_raw = ''.join(df.rdd.map(lambda row: row[0]).collect())
jobject = json.loads(json_raw)

# Create a new report based on the report.json file located in our Lakehouse
labs.report.create_report_from_reportjson(
    report = "Fabric REST API Docs", 
    dataset = semantic_model_name, 
    report_json = jobject, 
    workspace = workspace_name
    )